This file colects data and game metrics for every game scraped from GameRanks.ipynb

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
gameRanks=pd.read_csv("GameRanks.csv")
gameIds= gameRanks['ID']

Data Collection uses BoardGameGeek's XML API

In [3]:
rootpath = 'https://boardgamegeek.com/xmlapi2/'
gamepath = 'thing/'
playpath = 'plays/'

I loop through every game ID and get play data from Feb 25 to March 25
I wait 5 seconds between each call. There is a more efficent way if you call every desired ID at once, but I didn't think of that yet.

In [23]:
gamePlays=[]
for id in gameIds[0:300]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [ ]:
#took a break, now resuming
for id in gameIds[300:399]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [29]:
#Lost connection
#Restarting
for id in gameIds[399:]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

I add the play data to the dataframe and update the csv to save the data if my environment restarts

In [35]:
gameRanks['Recorded Plays Last Month']=gamePlays

gameRanks=gameRanks[['ID', 'Recorded Plays Last Month', 'Overall Rank', 'Abstract Rank',
                     'Customizable Rank', 'Thematic Rank', 'Family Rank', 'Children Rank',
                     'Party Rank', 'Strategy Rank', 'War Rank']]

In [37]:
#Update CSV so I don't lose everything
#gameRanks.to_csv('GameRanks.csv')

I now call the api through a different path to get statistics on the game. This time, I call every game at once using a comma seperated string.

In [47]:
##Now for Complexity, updated ranks, and wanting/wishing
gameparameters={"id": ",".join(map(str, gameIds)),
                'stats': 1}

r = requests.get(rootpath+gamepath, params=gameparameters)
r.status_code

200

In [48]:
soup = BeautifulSoup(r.text, features='xml')

I use regex and BeautifulSoup to get the desired information

In [54]:
names = re.findall('value="([^"]+)"', str(soup.find_all('name', {'type':'primary'})))

In [ ]:
gameRanks['Name']=names

In [64]:
ratings = re.findall('value="(\d+)"', str(soup.find_all('rank', {'name': 'boardgame'})))

In [79]:
gameRanks['Overall Rank']=ratings

In [106]:
complexity = re.findall('value="(\d\.*\d*)"', str(soup.find_all('averageweight')))
scores = re.findall('value="(\d\.*\d*)"', str(soup.find_all('bayesaverage')))
owned = re.findall('value="(\d+)"', str(soup.find_all('owned')))
wants = re.findall('value="(\d+)"', str(soup.find_all('wanting')))
wishes = re.findall('value="(\d+)"', str(soup.find_all('wishing')))

In [126]:
year = re.findall('value="(-?\d*)"', str(soup.find_all('yearpublished')))

In [128]:
gameRanks['Year']=year

C:\Users\caleb\AppData\Local\Temp\ipykernel_29712\4278219437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameRanks['Year']=year


In [129]:
gameRanks

,Name,ID,Rating,Complexity,Overall Rank,Plays Last Month,Owns,Wants,Wishlists,Abstract Rank,Customizable Rank,Thematic Rank,Family Rank,Children Rank,Party Rank,Strategy Rank,War Rank,Year
0,Brass: Birmingham,224517,8.41705,3.8836,1,1703,61495,1735,17615,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2018
1,Pandemic Legacy: Season 1,161936,8.38317,2.8317,2,823,82267,813,13567,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,2015
2,Gloomhaven,174430,8.36184,3.9072,3,1934,96001,1193,20184,NaN,NaN,2.0,NaN,NaN,NaN,4.0,NaN,2017
3,Ark Nova,342942,8.33346,3.7493,4,12463,60548,1044,12923,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2021
4,Twilight Imperium: Fourth Edition,233078,8.24315,4.3171,5,485,28412,1035,11288,NaN,NaN,3.0,NaN,NaN,NaN,5.0,NaN,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,The Devil's Cauldron: The Battles for Arnhem a...,15369,5.92187,3.8372,3350,3,910,140,447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,2008
747,Successors (Fourth Edition),325348,6.09386,4.0652,2432,21,2278,91,698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,2022
748,Polis,303551,6.20844,3.7869,1978,17,1894,101,764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,2020
749,Roads to Gettysburg II: Lee Strikes North,215565,5.85047,3.6667,3874,8,919,68,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,2018


In [93]:
gameRanks['Complexity']=complexity
gameRanks['Rating']=scores
gameRanks['Owns']=owned
gameRanks['Wants']=wants
gameRanks['Wishlists']=wishes

I rename the Plays column and reorder all columns in the data frame to write the final CSV.

In [118]:
gameRanks['Plays Last Month']=gameRanks['Recorded Plays Last Month']

In [130]:
gameRanks=gameRanks[['Name','Year', 'ID', 'Rating', 'Complexity', 'Overall Rank', 'Plays Last Month',
                     'Owns', 'Wants', 'Wishlists', 'Abstract Rank', 'Customizable Rank', 'Thematic Rank',
                     'Family Rank', 'Children Rank', 'Party Rank', 'Strategy Rank', 'War Rank']]

In [131]:
#Write the Final CSV
#gameRanks.to_csv('GameRanks.csv')